In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

import os
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'whitesideReference'
]

Using TensorFlow backend.


In [2]:
def one_axis_femur_model(name='PCLOrigin', axis=0):
    output_path = "/code/" + name + "/"

    data_size = 15
    imgs_to_process = []
    lbls_to_process = []
    dif_arr = [(0,1,2), (0,2,1), (1,0,2), (1,2,0), (2,0,1), (2,1,0)]

    for index in range(data_size):
        for a, b, c in dif_arr:
            imgs_to_process.append(np.load(output_path + 'images/' + '{}-{}-{}-{}.npy'.format(index, a, b, c)))
            label = np.load(output_path + 'label/' + '{}-{}-{}-{}.npy'.format(index, a, b, c))
            lbls_to_process.append(int(label[3+axis]))

    imgs_to_process = np.array(imgs_to_process)
    lbls_to_process = np.array(lbls_to_process)

    batch_size = 15
    epochs = 50
    learning_rate = 0.001

    x_train, x_test, y_train, y_test = train_test_split(imgs_to_process, lbls_to_process, test_size=0.2)
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    colors = x_train.shape[3]


    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1,colors, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1,colors, img_rows, img_cols)
        input_shape = (1, colors, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, colors, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, colors, 1)
        input_shape = (img_rows, img_cols, colors, 1)

    num_classes = 120
    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)


    model = Sequential()
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(1, 1, 1)))




    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3,3,3)))

    model.add(Flatten())
    # Fully connected layer

    BatchNormalization()
    model.add(Dense(512))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.2))
    model.add(Dense(120))

    model.add(Activation('softmax'))


    sgd=SGD(lr=learning_rate)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])


In [3]:
one_axis_femur_model('medialSulcus', 0)

x_train shape: (72, 120, 120, 120)
72 train samples
18 test samples
72 train samples
18 test samples
Train on 72 samples, validate on 18 samples
Epoch 1/50
72/72 [==============================] - 245s 3s/step - loss: 19.4401 - accuracy: 0.0833 - val_loss: 5.2391 - val_accuracy: 0.1111
Epoch 2/50
72/72 [==============================] - 244s 3s/step - loss: 3.9658 - accuracy: 0.2500 - val_loss: 5.1628 - val_accuracy: 0.0556
Epoch 3/50
72/72 [==============================] - 244s 3s/step - loss: 3.4723 - accuracy: 0.3194 - val_loss: 5.2043 - val_accuracy: 0.0000e+00
Epoch 4/50
72/72 [==============================] - 245s 3s/step - loss: 3.0237 - accuracy: 0.4306 - val_loss: 5.1491 - val_accuracy: 0.0556
Epoch 5/50
72/72 [==============================] - 247s 3s/step - loss: 0.9379 - accuracy: 0.7222 - val_loss: 9.1570 - val_accuracy: 0.1111
Epoch 24/50
72/72 [==============================] - 246s 3s/step - loss: 0.9974 - accuracy: 0.7500 - val_loss: 11.8609 - val_accuracy: 0.0556
Ep